In [15]:
import codepy.data_preparation as data_preparation
import codepy.build_model as build
import codepy.model_inspection as inspection
import codepy.predictions as predictions
import numpy as np
import random
import wandb
from wandb.keras import WandbCallback
# import database_pg

def initialize_wandb():
    wandb.init(project="ipynb_test_runs", entity="unilu-dstoolkits")
initialize_wandb()
num_classes, input_shape, x_train, y_train, x_test, y_test = data_preparation.prepare_data()
#Comment form here..
model = build.build_model(num_classes, input_shape)
def train_model(x_train, y_train, x_test, y_test, model):
    wandb.config = {
        "epochs": 15,
        "batch_size": 128,
        "loss": "categorical_crossentropy",
        "optimizer": "adam",
        "metrics" : "accuracy",
        "validation_split" : 0.1,
        
    }

    model.compile(loss= wandb.config['loss'], optimizer= wandb.config['optimizer'], metrics= [wandb.config['metrics']])

    model.fit(x_train, y_train, batch_size= wandb.config['batch_size'],
              epochs= wandb.config['epochs'], validation_split= wandb.config['validation_split'],
              validation_data=(x_test, y_test),
              callbacks=[WandbCallback()])

# Call function
train_model(x_train, y_train, x_test, y_test, model)
inspection.save_model(model)
#..To here for testing purposes 
loaded_model = inspection.load_model()
inspection.evaluate_loaded_model(x_test, y_test, loaded_model)
# predictions.predict_on_data(x_test, y_test, loaded_model)
# database_pg.create_milestone3_db()
# database_pg.create_input_pred_db()
# random_img_x, squeezed_random_img_x, img_from_db = database_pg.insert_load_random_image()
# database_pg.predict_and_persist(img_from_db, loaded_model)

    

2021-12-06 22:46:49.209348: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-12-06 22:46:49.209455: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples
Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_8 (Conv2D)           (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d_8 (MaxPooling  (None, 13, 13, 32)       0         
 2D)                                                             
                                                                 
 conv2d_9 (Conv2D)           (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_9 (MaxPooling  (None, 5, 5, 64)         0         
 2D)                                                             
                                                                 
 flatten_4 (Flatten)         (None, 1600)              0         
                                               

In [16]:
# Predicting something with loaded_model
prediction  = loaded_model.predict(x_test)
# print(prediction)

#convert to int array 
int_predictions = np.argmax(prediction, axis = 1)
print('Predictions array:', int_predictions, prediction.shape, type(x_test), x_test.shape)


Predictions array: [7 2 1 ... 4 5 6] (10000, 10) <class 'numpy.ndarray'> (10000, 28, 28, 1)


In [17]:

random_numbers = random.sample(range(0,10000),5)
print('Random Numbers selected in array:', random_numbers)

print('Showcase to evaluate predictions:')
for i, number in enumerate(random_numbers):
    wandb.log({"img": [wandb.Image(x_test[random_numbers[i]], 
                                   caption=int_predictions[random_numbers[i]])]})
    

Random Numbers selected in array: [1819, 3741, 4288, 8926, 2260]
Showcase to evaluate predictions:


In [18]:
wandb.finish()

accuracy,▁▇▇██
epoch,▁▃▅▆█
loss,█▂▂▁▁
val_accuracy,▁▅▆▆█
val_loss,█▄▂▂▁
accuracy,0.98156
best_epoch,4
best_val_loss,0.03682
epoch,4
loss,0.05985
val_accuracy,0.99017
